In [1]:
import os
from semantic_kernel.agents import AssistantAgentThread, ResponsesAgentThread, AzureAssistantAgent, AzureResponsesAgent
from azure.identity import EnvironmentCredential, get_bearer_token_provider
from dotenv import load_dotenv

load_dotenv()
pool_management_endpoint = os.getenv("POOL_MANAGEMENT_ENDPOINT")
ai_project_endpoint = os.getenv("AI_PROJECT_ENDPOINT")
chat_endpoint=os.getenv("OPENAI_ENDPOINT")
model_deployment_name=os.getenv("OPENAI_DEPLOYMENT_NAME")

## Semantic Kernel + AOAI Assistants API

In [2]:
# 1. Create the client using Azure OpenAI resources and configuration
provider = get_bearer_token_provider(EnvironmentCredential(), "https://cognitiveservices.azure.com/.default")
client = AzureAssistantAgent.create_client(
    endpoint=chat_endpoint, 
    ad_token_provider=provider, 
    deployment_name=model_deployment_name,
    api_version="2025-04-01-preview"
)

# 2. Create the assistant on the Azure OpenAI service
definition = await client.beta.assistants.create(
    model=model_deployment_name,
    name="ContextAssistant",
    instructions="Keep track of the context of the conversation and use it to answer the user's question.",
)

# 3. Create a Semantic Kernel agent for the Azure OpenAI assistant
agent = AzureAssistantAgent(client=client, definition=definition)

# 4. Create a new thread for use with the assistant
thread: AssistantAgentThread = AssistantAgentThread(client=client, messages=[
    {
        "role": "user",
        "content": "What is the capital of France?"
    },
    {
        "role": "assistant",
        "content": "The capital of France is Paris."
    }
])


print(f"# Thread: {[f'{msg.role}: {msg.content}' async for msg in thread.get_messages()]}")
TASK = "What about Canada?"
print(f"# User: {TASK}")
try:
    # 5. Invoke the agent for the current thread and print the response
    async for response in agent.invoke(messages=TASK, thread=thread):
        print(f"# Agent: {response}")
        thread = response.thread
finally:
    # 6. Clean up the resources
    await thread.delete() if thread else None
    await agent.client.beta.assistants.delete(agent.id)


# Thread: ['assistant: The capital of France is Paris.', 'user: What is the capital of France?']
# User: What about Canada?
# Agent: The capital of Canada is Ottawa.


## Semantic Kernel + AOAI Responses API

In [6]:
# 1. Create the client using Azure OpenAI resources and configuration
provider = get_bearer_token_provider(EnvironmentCredential(), "https://cognitiveservices.azure.com/.default")
client = AzureResponsesAgent.create_client(
    endpoint=chat_endpoint, 
    ad_token_provider=provider, 
    deployment_name=model_deployment_name,
    api_version="2025-04-01-preview"
)

# 2. Create a Semantic Kernel agent for the OpenAI Responses API
agent = AzureResponsesAgent(
    ai_model_id=model_deployment_name,
    client=client,
    instructions="Answer questions about from the user.",
    name="Joker",
)

USER_INPUTS = [
    "My name is John Doe.",
    "Tell me a joke",
    "Explain why this is funny.",
    "What have we been talking about?",
]

# 3. Create a thread for the agent
# If no thread is provided, a new thread will be
# created and returned with the initial response
thread = None

for user_input in USER_INPUTS:
    print(f"# User: '{user_input}'")
    # 4. Invoke the agent for the current message and print the response
    response = await agent.get_response(messages=user_input, thread=thread)
    print(f"# {response.name}: {response.content}")
    # 5. Update the thread so the previous response id is used
    thread = response.thread

# Continue with an existing thread id
thread = ResponsesAgentThread(client=client, previous_response_id=thread.id)
# 6. Ask the agent a new question to show the thread is still valid
new_user_input = "What is my name?"
print(f"# User: '{new_user_input}'")
response = await agent.get_response(messages=new_user_input, thread=thread)
print(f"# {response.name}: {response.content}")

# User: 'My name is John Doe.'
# Joker: Nice to meet you, John Doe! How can I assist you today?
# User: 'Tell me a joke'
# Joker: Sure! Here's one for you:

Why don't skeletons fight each other?  
Because they don't have the guts! 🎃
# User: 'Explain why this is funny.'
# Joker: Of course! Here's the breakdown:

The humor in this joke comes from wordplay and a double meaning. When we say "skeletons don't have the guts," the phrase serves as both a literal and figurative statement:

1. **Literal Meaning**: Skeletons are just bones—they don't have organs like "guts" (intestines or internal organs). So, skeletons physically lack the "guts" to do anything.

2. **Figurative Meaning**: The phrase "don't have the guts" is a common idiom that means lacking bravery or courage. By using this phrase in context with skeletons, the joke creates an amusing overlap between the literal and figurative interpretations.

The humor comes from combining these two ideas in an unexpected way!
# User: 'What ha